# Funciones

In [1]:
import os
import ast

import json
import yaml
import copy

import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
resultados_tune_deepfish = {
"tune":   {"name": "Deepfish_yolov8n-seg_AdamW"},
"tune2":  {"name": "Deepfish_yolov8n-seg_SGD"},
"tune3":  {"name": "Deepfish_LO_yolov8n-seg_AdamW"},
"tune4":  {"name": "Deepfish_LO_yolov8n-seg_SGD"},
"tune5":  {"name": "Deepfish_yolov8s-seg_AdamW"},
"tune6":  {"name": "Deepfish_yolov8s-seg_SGD"},
"tune7":  {"name": "Deepfish_LO_yolov8s-seg_AdamW"},
"tune8":  {"name": "Deepfish_LO_yolov8s-seg_SGD"},
"tune9":  {"name": "Deepfish_yolov8m-seg_AdamW"},
"tune10": {"name": "Deepfish_yolov8m-seg_SGD"},
"tune11": {"name": "Deepfish_LO_yolov8m-seg_AdamW"},
"tune12": {"name": "Deepfish_LO_yolov8m-seg_SGD"},
"tune13": {"name": "Deepfish_yolov8l-seg_AdamW"},
"tune14": {"name": "Deepfish_yolov8l-seg_SGD"},
"tune15": {"name": "Deepfish_LO_yolov8l-seg_AdamW"},
"tune16": {"name": "Deepfish_LO_yolov8l-seg_SGD"},
"tune17": {"name": "Deepfish_yolov8x-seg_AdamW"},
"tune18": {"name": "Deepfish_yolov8x-seg_SGD"},
"tune19": {"name": "Deepfish_LO_yolov8x-seg_AdamW"},
"tune20": {"name": "Deepfish_LO_yolov8x-seg_SGD"},
"tune21": {"name": "Deepfish_yolov9c-seg_AdamW"},
"tune22": {"name": "Deepfish_yolov9c-seg_SGD"},
"tune23": {"name": "Deepfish_LO_yolov9c-seg_AdamW"},
"tune24": {"name": "Deepfish_LO_yolov9c-seg_SGD"},
"tune25": {"name": "Deepfish_yolov9e-seg_AdamW"},
"tune26": {"name": "Deepfish_yolov9e-seg_SGD"},
"tune27": {"name": "Deepfish_LO_yolov9e-seg_AdamW"},
"tune28": {"name": "Deepfish_LO_yolov9e-seg_SGD"}
}

Funciones para recorrer los mejores runs de cada experimento y guardar sus resultados e hiperparámetros en un diccionario

In [3]:
def obtener_mejores_experimentos(directorio_todos_los_experimentos: str, diccionario_raytune: dict):
    """
    Actualiza el diccionario de resultados de RayTune con los dos mejores experimentos para cada tune,
    añadiendo sus IDs y métricas.

    Parámetros:
    -----------
    directorio_todos_los_experimentos : str
        Ruta base que contiene los experimentos organizados por tune.
    diccionario_raytune : dict
        Diccionario con información de cada tune, que se actualizará con los mejores experimentos.

    Retorno:
    --------
    None
    """
    for tune_number, tune_dict in diccionario_raytune.items():
        path_experimentos = os.path.join(directorio_todos_los_experimentos, tune_number)
        if not os.path.isdir(path_experimentos):
            continue
        procesar_tune(path_experimentos, tune_dict)


def procesar_tune(path_experimentos: str, tune_dict: dict):
    """
    Procesa un directorio de experimentos para un tune específico, actualizando el diccionario de resultados
    con el mejor experimento y sus métricas.

    Parámetros:
    -----------
    path_experimentos : str
        Ruta al directorio que contiene los resultados de un tune específico.
    tune_dict : dict
        Diccionario con información de un tune, que se actualizará con los mejores experimentos.

    Retorno:
    --------
    None
    """
    weights_path = os.path.join(path_experimentos, "weights", "last.pt")
    best_hyperparameters_path = os.path.join(path_experimentos, "best_hyperparameters.yaml")
    
    try:
        best_hyperparameters = retornar_mejores_hiperparametros(best_hyperparameters_path)
        best_puntaje = retornar_mejores_puntajes(best_hyperparameters_path)
        actualizar_diccionario_tune(tune_dict, best_hyperparameters, best_puntaje, weights_path)
    except FileNotFoundError:
        pass


def retornar_mejores_hiperparametros(best_hyperparameters_path: str) -> dict:
    """
    Lee los mejores hiperparámetros desde el archivo YAML obtenido y los retorna como un diccionario.
    
    Parámetros:
    -----------
    best_hyperparameters_path : str
        Ruta al archivo YAML que contiene los resultados de un tune específico.

    Retorno:
    --------
    dict
        Un diccionario con los mejores hiperparámetros del experimento.
    """
    with open(best_hyperparameters_path, 'r') as file:
        # Cargar el archivo YAML
        content = file.read()
        
        # Ignorar las líneas comentadas (comienzan con '#') y cargar solo los valores YAML
        yaml_content = "\n".join([line for line in content.splitlines() if not line.strip().startswith('#')])
        
        # Convertir el contenido YAML en un diccionario
        hyperparameters = yaml.safe_load(yaml_content)
        
    return hyperparameters


def retornar_mejores_puntajes(best_hyperparameters_path: str) -> dict:
    """
    Lee los mejores resultados desde los comentarios del archivo YAML obtenido y los retorna como un diccionario.
    
    Parámetros:
    -----------
    best_hyperparameters_path : str
        Ruta al archivo YAML que contiene los resultados de un tune específico.

    Retorno:
    --------
    dict
        Un diccionario con los resultados del experimento.
    """
    
    # Diccionario para almacenar los resultados extraídos
    resultados = {}
    
    # Abrir y leer el archivo línea por línea
    with open(best_hyperparameters_path, 'r') as file:
        for line in file:
            # Eliminar espacios en blanco y verificar si es un comentario
            line = line.strip()
            
            # Verificar si la línea contiene 'Best fitness metrics are' para obtener las métricas
            if 'Best fitness metrics are' in line:
                # Extraer el diccionario de métricas desde la línea de comentarios
                # La línea tiene la forma: "# Best fitness metrics are {<diccionario>}"
                # Usamos ast.literal_eval para convertir la parte del diccionario desde la línea de texto a un diccionario Python
                metrics_str = line.split('are', 1)[1].strip()
                metrics_dict = ast.literal_eval(metrics_str)
                
                # Extraer las métricas que nos interesan
                resultados['precision(M)'] = metrics_dict['metrics/precision(M)']
                resultados['recall(M)'] = metrics_dict['metrics/recall(M)']
                resultados['mAP50(M)'] = metrics_dict['metrics/mAP50(M)']
                resultados['mAP50-95(M)'] = metrics_dict['metrics/mAP50-95(M)']
                
                # Calcular F1 score
                precision = resultados['precision(M)']
                recall = resultados['recall(M)']
                f1_score = 2 * (precision * recall) / (precision + recall)
                resultados['F1_score'] = f1_score
                
                break  # No necesitamos seguir leyendo el archivo después de obtener las métricas
    
    return resultados


def actualizar_diccionario_tune(tune_dict: dict, best_experimento: dict, best_puntaje: dict, weights_path: str):
    """
    Actualiza el diccionario de resultados de un tune con los hiperparámetros del mejor entrenamiento y sus métricas.

    Parámetros:
    -----------
    tune_dict : dict
        Diccionario con información de un tune, que se actualizará con el mejor entrenamiento.
    best_experimento : dict
        Hiperparámetros del mejor entrenamiento.
    best_puntaje : dict
        Resultados del mejor entrenamiento.
    weights_path : str
        Localización del path

    Retorno:
    --------
    None
    """
    tune_dict.update(config=best_experimento)
    tune_dict.update(score=best_puntaje)


def guardar_diccionario_como_json(diccionario: dict, archivo_path: str):
    """
    Guarda un diccionario en un archivo JSON después de convertir pandas.Series a diccionarios.

    Parameters:
    -----------
    diccionario : dict
        Diccionario que contiene la información de los experimentos organizados por tune.
    archivo_path : str
        Ruta donde se guardará el archivo JSON.
    
    Returns:
    --------
    None
    """
    with open(archivo_path, 'w') as archivo:
        json.dump(diccionario, archivo, indent=4)

Copiar los pesos .pt y guardarlos en otro sitio para futuros entrenamientos y exportaciones

In [4]:
# obtener_mejores_experimentos("runs/segment", resultados_tune_deepfish)
# guardar_diccionario_como_json(resultados_tune_deepfish, 'resultados_tune_deepfish_1.json')